# KBGAT Source Code

# main.py

## Hàm `def load_data(args):`

### 1. build_data( args.data, is_unweigted=False, directed=True)

##### `train_data` : `(train_triples, train_adjacency_mat)` :

Trong đó : 

`train_triples` : `(entity2id[e1], relation2id[relation], entity2id[e2])`

`train_adjacency_mat` : `(rows, cols, data)`


`validation_data`, `test_data` : tương tự `train_data`

`entity2id`

`relation2id`

`headTailSelector` : 

`unique_entities_train`


`entity_embeddings` : shape = [len(entity2id) : args.embedding_size]

Có kích thước là [số lượng entity, số chiều k]

## Hàm def train_gat(args):

Định nghĩa mô hình 

#### SpKBGATModified()

`entity_embeddings`

`relation_embeddings`

`args.entity_out_dim`

`args.entity_out_dim`

`args.drop_GAT`

`args.alpha`

`args.nheads_GAT`

## Class SpKBGATModified:

`num_nodes`
`entity_in_dim`
`entity_out_dim_1`
`nheads_GAT_1`
`entity_out_dim_2`
`nheads_GAT_2`

#### Properties of Relations

`num_relation`
`relation_dim`
`relation_out_dim_1`
`drop_GAT`
`alpha`

#### Parameter
40943 : số phần tử entity trong tập train

11 : số phần tử relation trong tập train

[100, 200] : Entity output embedding dimensions (entity_out_dim)

[2, 2] : Multihead attention SpGAT (nheads_GAT)

50 : số chiều nhúng k ban đầu
```
final_entity_embeddings : torch.Size([40943, 200])
final_relation_embeddings : torch.Size([11, 200])
entity_embeddings : torch.Size([40943, 50])
relation_embeddings : torch.Size([11, 50])
W_entities : torch.Size([50, 200])
sparse_gat_1.W : torch.Size([50, 200])
sparse_gat_1.attention_0.a : torch.Size([100, 150])
sparse_gat_1.attention_0.a_2 : torch.Size([1, 100])
sparse_gat_1.attention_1.a : torch.Size([100, 150])
sparse_gat_1.attention_1.a_2 : torch.Size([1, 100])
sparse_gat_1.out_att.a : torch.Size([200, 600])
sparse_gat_1.out_att.a_2 : torch.Size([1, 200])
```


* `final_entity_embeddings` : shape = [num_nodes, entity_out_dim_1 * nheads_GAT_1]

(Kết quả nhúng thực thể : [số lượng nodes, kích thước output entity * số nheads_GAT_1])

* `final_relation_embeddings`: shape = [num_relation, entity_out_dim_1 * nheads_GAT_1]

* `entity_embeddings` : `initial_entity_emb`

* `relation_embeddings` : `initial_relation_emb`

* sparse_gat_1: SpGAT : tập hợp các paramter của lớp SpGAT

* `W_entities` : trọng số thực thể với shape = [entity_in_dim, entity_out_dim_1 * nheads_GAT_1]

## class SpGAT(nn.Module)
```python
SpKBGATModified(
  (sparse_gat_1): SpGAT(
    (dropout_layer): Dropout(p=0.3, inplace=False)
    (attention_0): SpGraphAttentionLayer (50 -> 100)
    (attention_1): SpGraphAttentionLayer (50 -> 100)
    (out_att): SpGraphAttentionLayer (200 -> 200)
  )
)
```


```python
sparse_gat_1.W : torch.Size([50, 200])
sparse_gat_1.attention_0.a : torch.Size([100, 150])
sparse_gat_1.attention_0.a_2 : torch.Size([1, 100])
sparse_gat_1.attention_1.a : torch.Size([100, 150])
sparse_gat_1.attention_1.a_2 : torch.Size([1, 100])
sparse_gat_1.out_att.a : torch.Size([200, 600])
sparse_gat_1.out_att.a_2 : torch.Size([1, 200])```


#### Dropout

`dropout_layer = nn.Dropout(self.dropout)`

torch.Size([40943, 200])

## Hàm def forward() trong layer SpGAT

Input : 
```
Corpus_
batch_inputs
entity_embeddings
relation_embed
edge_list
edge_type
edge_embed
edge_list_nhop
edge_type_nhop
```


## Hàm def forward() trong layer chính (SpKBGATModified)
Input :

`train_adj_matrix` : gồm hai phần tử `(adj_indices, adj_values)`
`train_indices` : lấy từ hàm `get_iteration_batch` : `batch_indices` : là index lấy ra của từng batch
`current_batch_2hop_indices` : Lấy 2 hop indices kế cận của batch


`edge_list` : head
`edge_type` : tailSpectrum of a bounded operator

input, edge, edge_embed, edge_list_nhop, edge_embed_nhop):




### model_conv
```python
final_entity_embeddings : torch.Size([40943, 200])
final_relation_embeddings : torch.Size([11, 200])
convKB.conv_layer.weight : torch.Size([500, 1, 1, 3])
convKB.conv_layer.bias : torch.Size([500])
convKB.fc_layer.weight : torch.Size([1, 100000])
convKB.fc_layer.bias : torch.Size([1])

```

```python
SpKBGATConvOnly(
  (convKB): ConvKB(
    (conv_layer): Conv2d(1, 500, kernel_size=(1, 3), stride=(1, 1))
    (dropout): Dropout(p=0.0, inplace=False)
    (non_linearity): ReLU()
    (fc_layer): Linear(in_features=100000, out_features=1, bias=True)
  )
)
```

# Debug

```python
entity_embeddings (40943, 50)
relation_embed(11, 50)


edge_list[i] (2, 86835) : head_id, tail_id của một cạnh i ()
edge_type[i] (86835) : relation_id của một cạnh i

edge_list_nhop[i] (2, 207376) : neigh_entity_id1, neigh_entity_id2 của một cạnh hàng xóm với source
edge_type[i] (2, 207376) : neigh_relation_id_1, neigh_relation_id_1

edge_embed (86835, 50)


edge_embed_nhop (207376, 50) = G_nhop1 + G_nhop2

train_indices_nhop (207376, 4)

N = 40943

___________________________________________
edge (2, 294211)
edge_embed (2, 294211) = edge_embed(86835, 50) + edge_embed_nhop(207376, 50)

edge_h (150, 294211)

A_1 (100, 150)
edge_m (100, 294211)


A_2 (1, 100)

powers (294211)
edge_e (294211, 1)



```